In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from datetime import timedelta
import time
import matplotlib.pyplot as plt
import csv
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
from moviepy.editor import VideoFileClip
import ffmpeg

## Step 1: Global inputs

In [2]:
videoRecreate = False # set to True, regenerate video when source video cannot be displayed

shotsAfter = 0 # number of shots to add after the end shot with recognized faces
interval = 30 # inverval in seconds, it determines the granuity of the clips

# Inputs are needed below
videosDir = ""
videoExt = ".mp4"
characterName = ""
outputExt = ".mp4"
opSeconds = 0 # opening song in seconds
edSeconds = 0 # ending song in seconds
interval = 5 # inverval in seconds, it determines the granuity of the clips

# create the directories to store intermediate and final results
knownFacesDir = videosDir + "/knownFaces"
attendanceDir = videosDir + "/attendance"
testOutputDir = videosDir + "/test" # dir to save images of recognized faces
statsDir = videosDir + "/scenes"
timeRangesDir = videosDir + "/time_ranges"
outputDir = videosDir + "/" + characterName

##  Step 2: Scene detect for series

In [3]:
for filename in os.listdir(videosDir):
    if filename.endswith(videoExt):
        video_name = os.path.splitext(filename)[0]
        video_file = os.path.join(videosDir, filename)
        stats_file = os.path.join(statsDir, video_name + "_stats.csv")
        !scenedetect -i "$video_file" list-scenes -s -f "$stats_file" detect-adaptive

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/22_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 999 | Progress: 100%|█████▉| 67348/67362 [08:20<00:00, 134.51frames/s]
[PySceneDetect] Processed 67348 frames in 500.7 seconds (average 134.50 FPS).
[PySceneDetect] Detected 1000 scenes, average shot length 2.7 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/22_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 

 |    808  |       53743 | 00:35:49.680 |       53841 | 00:35:53.640 |
 |    809  |       53842 | 00:35:53.640 |       53918 | 00:35:56.720 |
 |    810  |       53919 | 00:35:56.720 |       53946 | 00:35:57.840 |
 |    811  |       53947 | 00:35:57.840 |       53989 | 00:35:59.560 |
 |    812  |       53990 | 00:35:59.560 |       54019 | 00:36:00.760 |
 |    813  |       54020 | 00:36:00.760 |       54098 | 00:36:03.920 |
 |    814  |       54099 | 00:36:03.920 |       54157 | 00:36:06.280 |
 |    815  |       54158 | 00:36:06.280 |       54253 | 00:36:10.120 |
 |    816  |       54254 | 00:36:10.120 |       54291 | 00:36:11.640 |
 |    817  |       54292 | 00:36:11.640 |       54403 | 00:36:16.120 |
 |    818  |       54404 | 00:36:16.120 |       54450 | 00:36:18.000 |
 |    819  |       54451 | 00:36:18.000 |       54485 | 00:36:19.400 |
 |    820  |       54486 | 00:36:19.400 |       54516 | 00:36:20.640 |
 |    821  |       54517 | 00:36:20.640 |       54557 | 00:36:22.280 |
 |    

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/19_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 881 | Progress: 100%|█████▉| 70260/70269 [08:35<00:00, 136.25frames/s]
[PySceneDetect] Processed 70260 frames in 515.7 seconds (average 136.24 FPS).
[PySceneDetect] Detected 882 scenes, average shot length 3.2 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/19_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 |

 |    481  |       38239 | 00:25:29.520 |       38431 | 00:25:37.240 |
 |    482  |       38432 | 00:25:37.240 |       38613 | 00:25:44.520 |
 |    483  |       38614 | 00:25:44.520 |       38745 | 00:25:49.800 |
 |    484  |       38746 | 00:25:49.800 |       38796 | 00:25:51.840 |
 |    485  |       38797 | 00:25:51.840 |       38863 | 00:25:54.520 |
 |    486  |       38864 | 00:25:54.520 |       38914 | 00:25:56.560 |
 |    487  |       38915 | 00:25:56.560 |       38989 | 00:25:59.560 |
 |    488  |       38990 | 00:25:59.560 |       39098 | 00:26:03.920 |
 |    489  |       39099 | 00:26:03.920 |       39164 | 00:26:06.560 |
 |    490  |       39165 | 00:26:06.560 |       39217 | 00:26:08.680 |
 |    491  |       39218 | 00:26:08.680 |       39269 | 00:26:10.760 |
 |    492  |       39270 | 00:26:10.760 |       39476 | 00:26:19.040 |
 |    493  |       39477 | 00:26:19.040 |       39530 | 00:26:21.200 |
 |    494  |       39531 | 00:26:21.200 |       39623 | 00:26:24.920 |
 |    

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/14_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 1077 | Progress: 100%|████▉| 68660/68665 [08:25<00:00, 135.81frames/s]
[PySceneDetect] Processed 68660 frames in 505.6 seconds (average 135.81 FPS).
[PySceneDetect] Detected 1078 scenes, average shot length 2.5 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/14_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/11_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 981 | Progress: 100%|█████▉| 66979/66985 [08:27<00:00, 132.01frames/s]
[PySceneDetect] Processed 66979 frames in 507.4 seconds (average 132.01 FPS).
[PySceneDetect] Detected 982 scenes, average shot length 2.7 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/11_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 |

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/12_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 986 | Progress: 100%|█████▉| 68323/68332 [08:32<00:00, 133.27frames/s]
[PySceneDetect] Processed 68323 frames in 512.7 seconds (average 133.27 FPS).
[PySceneDetect] Detected 987 scenes, average shot length 2.8 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/12_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 |

[PySceneDetect] PySceneDetect 0.6.2
[PySceneDetect] Scene list filename format:
  莲花楼1080/scenes/13_stats.csv
[PySceneDetect] Downscale factor set to 15, effective resolution: 256 x 107
[PySceneDetect] Detecting scenes...                                             
Detected: 890 | Progress: 100%|█████▉| 68562/68566 [08:41<00:00, 131.52frames/s]
[PySceneDetect] Processed 68562 frames in 521.3 seconds (average 131.52 FPS).
[PySceneDetect] Detected 891 scenes, average shot length 3.1 seconds.
[PySceneDetect] Writing scene list to CSV file:
  莲花楼1080/scenes/13_stats.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           1 | 00:00:00.000 |         150 | 00:00:06.000 |
 |      2  |         151 | 00:00:06.000 |         275 | 00:00:11.000 |
 |      3  |         276 | 00:00:11.000 |

## Step 3: Detect Faces and Mark Attendance 

In [9]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

# TODO: allow multiple character names
def recognizeFaces(inputFile, attendanceDir, testOutputDir, \
                   classNames, characterName, encodeListKnown):
    
    cap = cv2.VideoCapture(inputFile)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        exit()
    
    # Use os.path.basename to extract the filename without the path
    filename = os.path.basename(inputFile)
    filename = filename.replace('/', '')
    videoName = os.path.splitext(filename)[0]
    attendanceFile = attendanceDir + "/" + videoName + ".csv"
    testOutput = testOutputDir + "/" + videoName
    
    if not os.path.exists(testOutput):
        os.makedirs(testOutput)

    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    original_fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original frames per second
    new_fps = 1      # Desired new frames per second

    if videoRecreate:
        # file extension hardcode here, only .mp4 is supported at this moment
        output_video_path = os.path.splitext(inputFile)[0] + ".mp4"
        # Define the codec and create VideoWriter object for the output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # hardcode here, Codec for the output video (MP4V for MP4)
        out = cv2.VideoWriter(output_video_path, fourcc, original_fps, frame_size)
    
    frame_skip_factor = original_fps / new_fps
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_counter = -1
    while True:
        frame_counter += 1
        # read frames

        ret, img = cap.read()
        # check if frame is None
        if img is None:
            #if True break the infinite loop
            break
            
        if videoRecreate:
            out.write(img)
            
        if frame_counter % frame_skip_factor != 0:
             continue

        imgS = cv2.resize(img,(0,0),None,0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

        seconds = frame_counter / frame_skip_factor

        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
            #print(faceDis)
            matchIndex = np.argmin(faceDis)

            # TODO: fine-tune threshold
            if faceDis[matchIndex]< 0.4:
                name = classNames[matchIndex].upper()
                if characterName in name:
                    markAttendance(name, seconds, attendanceFile)
                    if not os.path.exists(testOutputDir):
                        os.makedirs(testOutputDir)
        
                    y1, x2, y2, x1 = faceLoc
                    y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                    cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                    cv2.putText(img,f'{name} {round(faceDis[matchIndex],2)}',(x1+6,y2-6),\
                                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    cv2.imwrite( testOutput + "/" + \
                                str(timedelta(seconds=seconds)).replace(":", "m") + '.jpg', img)
                    break

            else:
                name = 'Unknown'
    
    cap.release()
    #out.release()
    
    

    cv2.destroyAllWindows()

def markAttendance(name, seconds, attendanceFile):
    # Define the header line
    header = "Name,Time In"

    with open(attendanceFile, 'a+') as f:
        # Check if the file is empty (no previous data)
        f.seek(0, 2)  # Go to the end of the file
        if f.tell() == 0:
            # If the file is empty, write the header
            f.write(header + '\n')

        time_in = timedelta(seconds=seconds)

        # Convert timedelta objects to strings
        time_in_str = str(time_in)

        # Append the attendance record
        f.write(f'{name},{time_in_str}\n')
        
def main():
    # Check if the folder exists; if not, create it
    if not os.path.exists(testOutputDir):
        os.makedirs(testOutputDir)
    if not os.path.exists(attendanceDir):
        os.makedirs(attendanceDir)
    
    print(type(characterName), characterName)
    
    images = []
    classNames = []
    myList = os.listdir(knownFacesDir)
    for cl in myList:
        if cl.startswith("."):
            continue
        curImg = cv2.imread(f'{knownFacesDir}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])

    encodeListKnown = findEncodings(images)
    
    i = 1
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            recognizeFaces(videosDir + "/" + filename, attendanceDir, testOutputDir, \
                           classNames, characterName, encodeListKnown)
            # TODO: handle error if next functions depend on the success run of recognizing Faces
            print("#" + str(i) + " processed")
            i = i + 1

In [10]:
if __name__ == "__main__":
    main()

## Step 4: Combine the results of facial recognition and scene detect

In [12]:
def read_time_spots(inputFile):
    # Define the time spots as a list of time strings
    file = open(inputFile, 'r') 
    csv1 = csv.reader(file,delimiter=',')
    next(csv1, None)
    time_spots = []

    for row in csv1:
        time_spots.append(row[1] + ".000")
    time_spots = [datetime.strptime(time_spot, "%H:%M:%S.%f") for time_spot in time_spots]
    
    return time_spots
    
def read_scene_time_ranges(inputFile):
    
    # Initialize the list to store time ranges
    time_ranges = []

    # Read the CSV file
    with open(inputFile, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Iterate through the CSV rows
        for row in csv_reader:
            start_time = timedelta(seconds = float(row['Start Time (seconds)']))
            start_time_str = str(start_time)
            end_time = timedelta(seconds = float(row['End Time (seconds)']))
            end_time_str = str(end_time)
            if '.' not in start_time_str:
                start_time_str += ".000"
            if '.' not in end_time_str:
                end_time_str += ".000"
            time_ranges.append((start_time_str, end_time_str))
            
    # Convert time strings to datetime objects for easy comparison
    time_ranges = [(datetime.strptime(str(start), "%H:%M:%S.%f"), datetime.strptime(str(end), "%H:%M:%S.%f")) for start, end in time_ranges]
    
    return time_ranges

def combine_time_ranges(time_ranges):
    if not time_ranges:
        return []
    
    # TODO: fine tune interval
    allow_interval_sec = timedelta(seconds = interval)

    sorted_ranges = sorted(time_ranges)
    combined_ranges = [sorted_ranges[0]]

    for start, end in sorted_ranges[1:]:
        last_start, last_end = combined_ranges[-1]

        if start <= last_end + allow_interval_sec:
            combined_ranges[-1] = (last_start, max(last_end, end))
        else:
            combined_ranges.append((start, end))

    return combined_ranges

def create_unique_time_ranges(time_spots, time_ranges, outputFile):
    # Create a set to store the unique time ranges
    unique_time_ranges = set()

    # Iterate through time spots and check if they fall within any time range
    for spot in time_spots:
        for i in range(len(time_ranges)):
            start, end = time_ranges[i]
            if start <= spot <= end:
                
                if i + shotsAfter < len(time_ranges):
                    end = time_ranges[i + shotsAfter][1]
                    
                unique_time_ranges.add((start, end))
                
                break  # No need to check other time ranges once a match is found

    # Sort and combine overlapping time ranges
    sorted_time_ranges = combine_time_ranges(unique_time_ranges)

    # Save the time ranges to the CSV file
    with open(outputFile, mode="w", newline="") as csv_file:
        fieldnames = ["start_time", "end_time"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write each time range as a row in the CSV file
        for start, end in sorted_time_ranges:
            writer.writerow({"start_time": start.strftime("%H:%M:%S.%f"), "end_time": end.strftime("%H:%M:%S.%f")})

        print(f"Time ranges have been saved to {outputFile}")
        
def main():
    if not os.path.exists(timeRangesDir):
        os.makedirs(timeRangesDir)
        
    # use videosDir here, with assumption that the .csv file names starts with videoName
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            filename = os.path.basename(filename)
            filename = filename.replace('/', '')
            videoName = os.path.splitext(filename)[0]
            
            time_spots_file = attendanceDir + "/" + videoName + ".csv"
            time_spots = read_time_spots(time_spots_file)
            
            time_ranges_file = statsDir + "/" + videoName + "_stats.csv"
            time_ranges = read_scene_time_ranges(time_ranges_file)
            
            outputFile = timeRangesDir + "/" + videoName + "_time_ranges.csv"
            create_unique_time_ranges(time_spots, time_ranges, outputFile)
    
if __name__ == "__main__":
    main()

Time ranges have been saved to 莲花楼1080/time_ranges/22_time_ranges.csv
Time ranges have been saved to 莲花楼1080/time_ranges/19_time_ranges.csv
Time ranges have been saved to 莲花楼1080/time_ranges/14_time_ranges.csv
Time ranges have been saved to 莲花楼1080/time_ranges/11_time_ranges.csv
Time ranges have been saved to 莲花楼1080/time_ranges/12_time_ranges.csv
Time ranges have been saved to 莲花楼1080/time_ranges/13_time_ranges.csv


## Step 5: Get video clips by time range

In [3]:
def get_sec(time_str):
    """Get Seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

def get_video_duration(video_path):
    try:
        video = VideoFileClip(video_path)
        duration = video.duration
        return duration
    except Exception as e:
        print(f"Error: {e}")
        return None


def main():
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)
    
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            filePath = videosDir + "/" + filename
            
            filename = os.path.basename(filename)
            filename = filename.replace('/', '')
            videoName = os.path.splitext(filename)[0]
            
            videoDuration = get_video_duration(filePath)
            
            opTime = opSeconds
            edTime = videoDuration - edSeconds
            
            time_ranges_file = timeRangesDir + "/" + videoName + "_time_ranges.csv"
            sample = open(time_ranges_file, 'r') 
            csv1 = csv.reader(sample,delimiter=',')
            next(csv1, None)
            n = 1
            
            # Create a list to store video clips
            video_clips = []
            
            for row in csv1:
                start = get_sec(row[0])
                end = get_sec(row[1])
                if start >= edTime or end <= opTime:
                    continue
                
                if start < edTime and end > opTime:
                    start = max(start, opTime)
                    end = min(end, edTime)
                    
                out_file = outputDir + "/" + videoName + '_' + str(n) + outputExt
                print(filePath, start, end, out_file)
                ffmpeg_extract_subclip(filePath, start, end, targetname=out_file)
                n = n + 1

if __name__ == "__main__":
    main()

莲花楼1080/22.mp4 188.6 193.12 莲花楼1080/李莲花（李相夷）/22_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 311.12 314.92 莲花楼1080/李莲花（李相夷）/22_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 325.16 348.6 莲花楼1080/李莲花（李相夷）/22_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 370.8 386.2 莲花楼1080/李莲花（李相夷）/22_4.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 407.12 437.12 莲花楼1080/李莲花（李相夷）/22_5.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 450.24 533.64 莲花楼1080/李莲花（李相夷）/22_6.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 543.84 587.84 莲花楼1080/李莲花（李相夷）/22_7.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp4 598.36 628.8 莲花楼1080/李莲花（李相夷）/22_8.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/22.mp

Moviepy - Command successful
莲花楼1080/19.mp4 2579.28 2581.0 莲花楼1080/李莲花（李相夷）/19_30.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/19.mp4 2595.08 2641.12 莲花楼1080/李莲花（李相夷）/19_31.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 104 157.56 莲花楼1080/李莲花（李相夷）/14_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 167.84 170.04 莲花楼1080/李莲花（李相夷）/14_2.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 211.56 215.68 莲花楼1080/李莲花（李相夷）/14_3.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 246.76 268.32 莲花楼1080/李莲花（李相夷）/14_4.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 461.84000000000003 464.84000000000003 莲花楼1080/李莲花（李相夷）/14_5.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/14.mp4 478.52 480.48 莲花楼1080/李莲花（李相夷）/14_6.mp4
Moviepy - Running:
>>>

Moviepy - Command successful
莲花楼1080/11.mp4 486.24 487.96 莲花楼1080/李莲花（李相夷）/11_7.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 505.8 574.44 莲花楼1080/李莲花（李相夷）/11_8.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 584.88 761.56 莲花楼1080/李莲花（李相夷）/11_9.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 778.92 871.72 莲花楼1080/李莲花（李相夷）/11_10.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 889.16 978.48 莲花楼1080/李莲花（李相夷）/11_11.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 999.92 1068.44 莲花楼1080/李莲花（李相夷）/11_12.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 1116.2 1118.28 莲花楼1080/李莲花（李相夷）/11_13.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/11.mp4 1130.56 1213.16 莲花楼1080/李莲花（李相夷）/11_14.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)


Moviepy - Command successful
莲花楼1080/12.mp4 1549.8 1561.92 莲花楼1080/李莲花（李相夷）/12_33.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1580.4 1587.0 莲花楼1080/李莲花（李相夷）/12_34.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1607.64 1609.56 莲花楼1080/李莲花（李相夷）/12_35.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1622.8 1624.44 莲花楼1080/李莲花（李相夷）/12_36.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1635.4 1734.4 莲花楼1080/李莲花（李相夷）/12_37.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1745.52 1765.08 莲花楼1080/李莲花（李相夷）/12_38.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1784.4 1834.56 莲花楼1080/李莲花（李相夷）/12_39.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
莲花楼1080/12.mp4 1848.2 1860.48 莲花楼1080/李莲花（李相夷）/12_40.mp4
Moviepy - Running:
>>> "+ " ".jo